### Calculation of SOH for the VW Measurements

In [1]:
import os
import sys
import math
sys.path.append(os.path.join(os.getcwd().partition('Code')[0], "Code"))
from src.config_base import GeneralConfig

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patches as mpatches
import matplotlib.text as mtext

In [3]:
from src.data.read_pickle import ReadPickle
from src.filtering.filter_methods import FilterMethods
from src.voltage_capacity_analysis.dva import DVA
from src.visualization.config_visualization import VisualizationConfig
from src.visualization.colormaps import ColorMaps

In [4]:
tum_orange_cmap = ColorMaps.blue_orange_tum()

In [5]:
colors_pack = tum_orange_cmap(np.linspace(0,1, 7))

In [6]:
colors_pack = np.vstack([colors_pack,np.array([156/255,157/255,159/255,1])])

In [7]:
def cm2inch(value):
    return value/2.54

In [8]:
nominal_capacity_cell = 78 # for vw id3 cell it is 2*78Ah=156Ah

In [9]:
net_energy = 58_000 #kWh vehicle COC
gross_energy = 62_000 #kWh vehicle COC

# Read Cell Files

In [10]:
path2files_cell = os.path.join(GeneralConfig.path2data.value,"VW","cell")

In [11]:
read_pickle = ReadPickle()

In [12]:
df_cell = read_pickle.read(os.path.join(path2files_cell,"VW_LG_78Ah_NMC_20deg_CP_C45.pkl"))

In [13]:
def print_infos_full_signal_cell(df):
    cell_v_max = df["U"].max()
    cell_v_min = df["U"].min()
    print("Voltage:")
    print(f"Max. voltage: {cell_v_max:.3f} V")
    print(f"Min. voltage: {cell_v_min:.3f} V")
    cell_soc_max = df["SOC"].max()
    cell_soc_min = df["SOC"].min()
    print("SOC:")
    print(f"Max. SOC: {cell_soc_max:.2f} %")
    print(f"Min. SOC: {cell_soc_min:.2f} %")
    cell_charge_max = df["Q"].max()
    power = df["I"]*(df["U"])
    norm_time = df["time_h"]-df["time_h"].iloc[0]
    energy =np.trapz(y=power, x=norm_time)
    print("Capacity:")
    print(f"Max. capacity: {cell_charge_max:.2f} Ah")
    print(f"Max. energy: {energy:.2f} Wh")
    s=108
    p=2
    print("SOH")
    print(f"SOH_cap: {cell_charge_max/(nominal_capacity_cell)*100:.2f} %")
    print(f"SOH_e_net: {energy*(p*s)/(net_energy)*100:.2f} %")
    print(f"SOH_e_gross: {energy*(p*s)/(gross_energy)*100:.2f} %")
    return cell_charge_max

In [14]:
print_infos_full_signal_cell(df_cell)

Voltage:
Max. voltage: 4.200 V
Min. voltage: 2.794 V
SOC:
Max. SOC: 100.00 %
Min. SOC: 0.00 %
Capacity:
Max. capacity: 80.26 Ah
Max. energy: 299.82 Wh
SOH
SOH_cap: 102.89 %
SOH_e_net: 111.66 %
SOH_e_gross: 104.45 %


80.2562514811762

# Read Vehicle Files

In [15]:
# nominal capacity estimated from initial VW measurement 2021
nominal_capacity_vehicle_v_limits = 75
print(nominal_capacity_vehicle_v_limits)

75


In [16]:
def print_infos_full_signal_vehicle(df,s=108,p=2,Q_offset = 0,E_offset=0):
    cell_v_max = df["U"].max()
    cell_v_min = df["U"].min()
    print("Voltage:")
    print(f"Max. voltage: {cell_v_max:.3f} V")
    print(f"Min. voltage: {cell_v_min:.3f} V")
    cell_soc_max = df["SOC"].max()
    cell_soc_min = df["SOC"].min()
    print("SOC:")
    print(f"Max. SOC: {cell_soc_max:.2f} %")
    print(f"Min. SOC: {cell_soc_min:.2f} %")
    cell_charge_max = df["Q"].max()+Q_offset
    power = df["I"]*(df["U"])
    norm_time = df["time_h"]-df["time_h"].iloc[0]
    energy =np.trapz(y=power, x=norm_time)+E_offset
    print("Capacity:")
    print(f"Max. capacity: {cell_charge_max:.2f} Ah")
    print(f"Max. energy: {energy/1000:.2f} kWh")
    print("SOH")
    print(f"SOH_cap: {cell_charge_max/(p*nominal_capacity_vehicle_v_limits)*100:.2f} %")
    print(f"SOH_e_net: {energy/(net_energy)*100:.2f} %")
    print(f"SOH_e_gross: {energy/(gross_energy)*100:.2f} %")
    return cell_charge_max

In [17]:
vehicle_charges = []

In [18]:
path2files_vw = os.path.join(GeneralConfig.path2data.value,"VW","vehicle")

In [19]:
def filter_func_preprocess_v_U(signal):
    perc_filter = 1 #% filter
    filtered_signal = FilterMethods().rolling_mean_df(signal,window_size=FilterMethods().round_to_next_odd_number(perc_filter/100 * len(signal)))
    #filtered_signal = FilterMethods().savgol(signal,window_size=FilterMethods().round_to_next_odd_number(perc_filter/100 * len(signal)))
    return filtered_signal

In [20]:
def filter_func_preprocess_v_Q(signal):
    perc_filter = 1 #% filter
    filtered_signal = FilterMethods().rolling_mean_df(signal,window_size=FilterMethods().round_to_next_odd_number(perc_filter/100 * len(signal)))
    #filtered_signal = FilterMethods().savgol(signal,window_size=FilterMethods().round_to_next_odd_number(perc_filter/100 * len(signal)))
    return filtered_signal

In [21]:
read_pickle = ReadPickle()
read_pickle.set_filter_U(filter_func_preprocess_v_U)
#read_pickle.set_filter_Q(filter_func_preprocess_v_Q)

In [22]:
df_vw =  read_pickle.read(os.path.join(path2files_vw,"VW_ID3_JB_8A_C40_2021.pkl"))
vehicle_charges.append(print_infos_full_signal_vehicle(df_vw,p=2))

Voltage:
Max. voltage: 451.574 V
Min. voltage: 359.304 V
SOC:
Max. SOC: 95.60 %
Min. SOC: 0.01 %
Capacity:
Max. capacity: 149.54 Ah
Max. energy: 60.57 kWh
SOH
SOH_cap: 99.69 %
SOH_e_net: 104.44 %
SOH_e_gross: 97.70 %


In [23]:
df_vw_aged =  read_pickle.read(os.path.join(path2files_vw,"VW_ID3_JB_8A_C40_2023.pkl"))
vehicle_charges.append(print_infos_full_signal_vehicle(df_vw_aged,p=2))

Voltage:
Max. voltage: 446.195 V
Min. voltage: 362.500 V
SOC:
Max. SOC: 96.00 %
Min. SOC: 0.00 %
Capacity:
Max. capacity: 136.56 Ah
Max. energy: 55.04 kWh
SOH
SOH_cap: 91.04 %
SOH_e_net: 94.90 %
SOH_e_gross: 88.78 %


In [24]:
12.1-3.5

8.6

In [25]:
df_vw_aged_plus =  read_pickle.read(os.path.join(path2files_vw,"VW_ID3_JB_8A_C40_2023.pkl")) # plus cathode capacity
E_offset_estimate = (8.6)*np.mean([454,446]) # 3.7*108 # 3.7V is to low for this high voltage regime. 
vehicle_charges.append(print_infos_full_signal_vehicle(df_vw_aged_plus,p=2,Q_offset = 8.6,E_offset=E_offset_estimate))

Voltage:
Max. voltage: 446.195 V
Min. voltage: 362.500 V
SOC:
Max. SOC: 96.00 %
Min. SOC: 0.00 %
Capacity:
Max. capacity: 145.16 Ah
Max. energy: 58.91 kWh
SOH
SOH_cap: 96.77 %
SOH_e_net: 101.58 %
SOH_e_gross: 95.02 %


In [27]:
df_vw_aged_su =  pd.read_feather(os.path.join(path2files_vw,"VW_ID3_JB_8A_C40_2024.feather"))
vehicle_charges.append(print_infos_full_signal_vehicle(df_vw_aged_su,p=2))

Voltage:
Max. voltage: 454.500 V
Min. voltage: 361.500 V
SOC:
Max. SOC: 96.00 %
Min. SOC: 0.00 %
Capacity:
Max. capacity: 146.14 Ah
Max. energy: 59.44 kWh
SOH
SOH_cap: 97.43 %
SOH_e_net: 102.49 %
SOH_e_gross: 95.88 %
